In [31]:
from sklearn.datasets import load_iris
import pandas as pd
from sklearn.model_selection import GridSearchCV,train_test_split,RandomizedSearchCV
data = load_iris()

In [21]:
#print(data.data)
#print(data.target)

#change to dataframe
data_df = pd.DataFrame(data.data)
df_y = pd.DataFrame(data.target,columns = ["target"])



Index(['target'], dtype='object')

In [28]:
model_df = pd.concat([data_df,df_y],axis = 1,ignore_index = True)

model_df.columns = ['col1','col2','col3','col4','target']

print(model_df.info())
print(model_df.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   col1    150 non-null    float64
 1   col2    150 non-null    float64
 2   col3    150 non-null    float64
 3   col4    150 non-null    float64
 4   target  150 non-null    int32  
dtypes: float64(4), int32(1)
memory usage: 5.4 KB
None
   col1  col2  col3  col4  target
0   5.1   3.5   1.4   0.2       0
1   4.9   3.0   1.4   0.2       0
2   4.7   3.2   1.3   0.2       0
3   4.6   3.1   1.5   0.2       0
4   5.0   3.6   1.4   0.2       0


In [32]:
model_df_raw = model_df.drop('target',axis = 1)

x = model_df_raw

y = model_df[['target']]

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size =0.1)

In [34]:
import lightgbm as lgb
lgb_train = lgb.Dataset(x_train, y_train)
lgb_test = lgb.Dataset(x_test, y_test, reference=lgb_train)

In [35]:
params = {
    'num_leaves': 5,
    'metric': ['l1', 'l2'],
    'verbose': -1
}

In [36]:
evals_result = {}  # to record eval results for plotting
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=100,
                valid_sets=[lgb_train, lgb_test],
                feature_name=[f'f{i + 1}' for i in range(x_train.shape[-1])],
                categorical_feature=[21],
                evals_result=evals_result,
                verbose_eval=10)

[10]	training's l1: 0.276046	training's l2: 0.108767	valid_1's l1: 0.322674	valid_1's l2: 0.150545
[20]	training's l1: 0.137632	training's l2: 0.0346031	valid_1's l1: 0.164616	valid_1's l2: 0.0563025
[30]	training's l1: 0.0851838	training's l2: 0.0230692	valid_1's l1: 0.107419	valid_1's l2: 0.0422813
[40]	training's l1: 0.0685608	training's l2: 0.0207402	valid_1's l1: 0.0931802	valid_1's l2: 0.0393895
[50]	training's l1: 0.0677401	training's l2: 0.0196244	valid_1's l1: 0.0930377	valid_1's l2: 0.0387906
[60]	training's l1: 0.0687042	training's l2: 0.0185439	valid_1's l1: 0.0927123	valid_1's l2: 0.0376843
[70]	training's l1: 0.0684068	training's l2: 0.0178545	valid_1's l1: 0.0927701	valid_1's l2: 0.039019
[80]	training's l1: 0.0684616	training's l2: 0.0171469	valid_1's l1: 0.0919427	valid_1's l2: 0.0391645
[90]	training's l1: 0.0679481	training's l2: 0.0165015	valid_1's l1: 0.0927977	valid_1's l2: 0.039914
[100]	training's l1: 0.0678893	training's l2: 0.0160162	valid_1's l1: 0.093596	val

C:\Users\lucaschang\Anaconda3\lib\site-packages\lightgbm\basic.py:1554: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [21]
  warnings.warn('categorical_feature in Dataset is overridden.\n'
C:\Users\lucaschang\Anaconda3\lib\site-packages\lightgbm\basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
C:\Users\lucaschang\Anaconda3\lib\site-packages\lightgbm\basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


In [37]:
def render_metric(metric_name):
    ax = lgb.plot_metric(evals_result, metric=metric_name, figsize=(10, 5))
    plt.show()

In [41]:
import matplotlib.pyplot as plt
%matplotlib inline

try:
    # To enable interactive mode you should install ipywidgets
    # https://github.com/jupyter-widgets/ipywidgets
    from ipywidgets import interact, SelectMultiple
    INTERACTIVE = True
except ImportError:
    INTERACTIVE = False

In [42]:
if INTERACTIVE:
    # create widget to switch between metrics
    interact(render_metric, metric_name=params['metric'])
else:
    render_metric(params['metric'][0])

interactive(children=(Dropdown(description='metric_name', options=('l1', 'l2'), value='l1'), Output()), _dom_c…

In [43]:
def render_plot_importance(importance_type, max_features=10,
                           ignore_zero=True, precision=3):
    ax = lgb.plot_importance(gbm, importance_type=importance_type,
                             max_num_features=max_features,
                             ignore_zero=ignore_zero, figsize=(12, 8),
                             precision=precision)
    plt.show()

In [45]:
if INTERACTIVE:
    # create widget for interactive feature importance plot
    interact(render_plot_importance,
             importance_type=['split', 'gain'],
             max_features=(1, x_train.shape[-1]),
             precision=(0, 10))
else:
    render_plot_importance(importance_type='split')

interactive(children=(Dropdown(description='importance_type', options=('split', 'gain'), value='split'), IntSl…

##plot split value histogram

In [46]:
def render_histogram(feature):
    ax = lgb.plot_split_value_histogram(gbm, feature=feature,
                                        bins='auto', figsize=(10, 5))
    plt.show()

In [47]:
if INTERACTIVE:
    # create widget for interactive split value histogram
    interact(render_histogram,
             feature=gbm.feature_name())
else:
    render_histogram(feature='f26')

interactive(children=(Dropdown(description='feature', options=('f1', 'f2', 'f3', 'f4'), value='f1'), Output())…

#plot trees

In [56]:
import graphviz
def render_tree(tree_index, show_info, precision=3):
    show_info = None if 'None' in show_info else show_info
    return lgb.create_tree_digraph(gbm, tree_index=tree_index,
                                   show_info=show_info, precision=precision)

In [57]:
if INTERACTIVE:
    # create widget to switch between trees and control info in nodes
    interact(render_tree,
             tree_index=(0, gbm.num_trees() - 1),
             show_info=SelectMultiple(  # allow multiple values to be selected
                 options=['None',
                          'split_gain',
                          'internal_value',
                          'internal_count',
                          'internal_weight',
                          'leaf_count',
                          'leaf_weight',
                          'data_percentage'],
                 value=['None']),
             precision=(0, 10))
    tree = None
else:
    tree = render_tree(53, ['None'])
tree


interactive(children=(IntSlider(value=49, description='tree_index', max=99), SelectMultiple(description='show_…